In [9]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.distributed as dist
import torch.utils.data as data_utils


class Net(nn.Module):
    def __init__(self, n_input, n_hidden, n_output):

        super(Net, self).__init__()
        self.hidden1 = nn.Linear(n_input, n_hidden)
        self.hidden2 = nn.Linear(n_hidden, n_hidden)

        self.predict1 = nn.Linear(n_hidden*2, n_output)
        self.predict2 = nn.Linear(n_hidden*2, n_output)

    def forward(self, input1, input2): # 多输入！！！
        input1 = input1.view(30,3)
        out01 = self.hidden1(input1)
        out02 = torch.relu(out01)
        out03 = self.hidden2(out02)
        out04 = torch.sigmoid(out03)

        input2 = input2.view(30,3)
        out11 = self.hidden1(input2)
        out12 = torch.relu(out11)
        out13 = self.hidden2(out12)
        out14 = torch.sigmoid(out13)

        out = torch.cat((out04, out14), dim=1) # 模型层拼合！！！当然你的模型中可能不需要~
 
        out = self.predict1(out)
        return out
#         out1 = self.predict1(out)
#         out2 = self.predict2(out)

#         return out1, out2 # 多输出！！！


net = Net(3, 4, 1)


x1 = torch.unsqueeze(torch.linspace(-1, 1, 90), dim=1) # 请不要关心这里，随便弄一个数据，为了说明问题而已
y1 = x1.pow(3)+0.1*torch.randn(x1.size())

x2 = torch.unsqueeze(torch.linspace(-1, 1, 90), dim=1)
y2 = x2.pow(3)+0.1*torch.randn(x2.size())

# x1, y1 = (Variable(x1), Variable(y1))
# x2, y2 = (Variable(x2), Variable(y2))

optimizer = torch.optim.SGD(net.parameters(), lr=0.1)
loss_func = torch.nn.MSELoss()

In [10]:
y1 = y1[:30]
# y1.size()
y2 = y2[:30]
# y2.size()
# x1

In [11]:
# y2.type()

# # print(x1.grad)
# x1.requires_grad = True
# # x1.requires_grad
# # loss1 = (x1**2+1).mean()
# # loss1.zero_grad
# # print(loss1)
# loss1 = (x1**2+1).mean()
# loss1.backward()
# # loss1.zero_grad()
# print(x1.grad)
# x1.grad.zero_()

In [12]:
for t in range(2):
#     prediction1, prediction2 = net(x1, x2)
#     loss1 = loss_func(prediction1, y1)
#     loss2 = loss_func(prediction2, y2)
#     loss = loss1 + loss2 # 重点！
    prediction1 = net(x1, x2)
    loss = loss_func(prediction1, y1)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if t % 1 == 0:
        print('Loss = %.4f' % loss.data)
        for name, parms in net.named_parameters():
            print('-->name:', name, '-->grad_requirs:',parms.requires_grad, ' \n-->grad_value:\n',parms.grad)
    
#     if t % 1 == 0:
#         print('Loss1 = %.4f' % loss1.data,'Loss2 = %.4f' % loss2.data,)
#         for name, parms in net.named_parameters():
#             print('-->name:', name, '-->grad_requirs:',parms.requires_grad, ' \n-->grad_value:\n',parms.grad)

Loss = 0.0978
-->name: hidden1.weight -->grad_requirs: True  
-->grad_value:
 tensor([[ 0.0055,  0.0053,  0.0051],
        [ 0.0009,  0.0009,  0.0009],
        [-0.0012, -0.0012, -0.0013],
        [ 0.0034,  0.0033,  0.0032]])
-->name: hidden1.bias -->grad_requirs: True  
-->grad_value:
 tensor([-0.0079,  0.0011, -0.0014, -0.0036])
-->name: hidden2.weight -->grad_requirs: True  
-->grad_value:
 tensor([[ 7.9900e-04, -6.0455e-04, -2.4897e-05,  5.3192e-05],
        [-6.2025e-03,  4.7794e-03,  1.9684e-04, -4.1012e-04],
        [-2.1973e-04,  1.6667e-04,  6.8495e-06, -1.4609e-05],
        [ 3.8799e-04, -2.9517e-04, -1.2190e-05,  2.5867e-05]])
-->name: hidden2.bias -->grad_requirs: True  
-->grad_value:
 tensor([ 0.0011, -0.0085, -0.0003,  0.0005])
-->name: predict1.weight -->grad_requirs: True  
-->grad_value:
 tensor([[0.0552, 0.0634, 0.0502, 0.0570, 0.0552, 0.0634, 0.0502, 0.0570]])
-->name: predict1.bias -->grad_requirs: True  
-->grad_value:
 tensor([0.1041])
-->name: predict2.weight -

In [88]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class FLModel(nn.Module):
    def __init__(self):
        super(FLModel, self).__init__()
        # 26.定义①的卷积层，输入为32x32的图像，卷积核大小5x5卷积核种类6
        self.conv1 = nn.Conv2d(1, 6, 2)
        # 27.定义③的卷积层，输入为前一层6个特征，卷积核大小5x5，卷积核种类16
        self.conv2 = nn.Conv2d(6, 16, 2)
        # 28.定义⑤的全链接层，输入为16*5*5，输出为120
        self.fc1 = nn.Linear(16, 120)  # 6*6 from image dimension
        # 29.定义⑥的全连接层，输入为120，输出为84
        self.fc2 = nn.Linear(120, 84)
        # 30.定义⑥的全连接层，输入为84，输出为10
        self.fc3 = nn.Linear(84, 14)
    def forward(self, x):
        t3 = torch.zeros(256,2)
        #print(x.size(),t3.size())
        x = torch.cat((x, t3), 1)
        x = x.view(256,1,9,9)
        # 31.完成input-S2，先卷积+relu，再2x2下采样
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # 32.完成S2-S4，先卷积+relu，再2x2下采样
        x = F.max_pool2d(F.relu(self.conv2(x)), 2) #卷积核方形时，可以只写一个维度
        # 33.将特征向量扁平成行向量
        x = x.view(-1, 16)
        # 34.使用fc1+relu
        x = F.relu(self.fc1(x))
        # 35.使用fc2+relu
        x = F.relu(self.fc2(x))
        # 36.使用fc3
        x = self.fc3(x)
#         return x
        output = F.log_softmax(x, dim=1)
        return output


net = FLModel()
print(net)

import numpy as np
# 打印网络的参数
params = list(net.parameters())
# print(params)
print(len(params))
# 打印某一层参数的形状
print(params[0].size())

#随机输入一个向量，查看前向传播输出
input = torch.randn(256,79)
# input = np.expand_dims(input, axis=2)
# print((input))
out = net(input)
print(out)

FLModel(
  (conv1): Conv2d(1, 6, kernel_size=(2, 2), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(2, 2), stride=(1, 1))
  (fc1): Linear(in_features=16, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=14, bias=True)
)
10
torch.Size([6, 1, 2, 2])
tensor([[-2.6457, -2.7180, -2.6952,  ..., -2.5762, -2.6698, -2.6415],
        [-2.6242, -2.6836, -2.6679,  ..., -2.5679, -2.6864, -2.6482],
        [-2.6299, -2.6830, -2.6731,  ..., -2.5654, -2.6906, -2.6618],
        ...,
        [-2.6286, -2.6972, -2.6643,  ..., -2.5779, -2.6780, -2.6510],
        [-2.6307, -2.7015, -2.6931,  ..., -2.5712, -2.6843, -2.6566],
        [-2.6327, -2.7043, -2.6622,  ..., -2.5577, -2.6874, -2.6625]],
       grad_fn=<LogSoftmaxBackward>)


In [99]:
x = torch.randn(64,79)
t3 = torch.zeros(256,2)
t4 = torch.zeros(64,2)
t5 = torch.zeros(x.size()[0],2)
x = torch.cat((x, t5), 1)
#print(x.size(),t3.size())
# try:
#     x = torch.cat((x, t3), 1)
# except:
#     x = torch.cat((x, t4), 1)
x

tensor([[-1.1486,  1.4084, -0.2635,  ..., -0.4677,  0.0000,  0.0000],
        [-0.0549, -1.0925, -1.6236,  ...,  0.1509,  0.0000,  0.0000],
        [-1.4069, -1.4782,  0.8286,  ...,  0.4287,  0.0000,  0.0000],
        ...,
        [-0.2381, -0.6959, -0.6199,  ...,  2.3516,  0.0000,  0.0000],
        [ 1.2532,  0.6781, -1.8472,  ..., -1.1791,  0.0000,  0.0000],
        [-0.2224, -0.3319,  0.8432,  ...,  1.4974,  0.0000,  0.0000]])

In [28]:
5632/128

44.0

In [52]:
32*32

1024

In [8]:
64*4*4

1024

In [58]:
import numpy as np
import torch

a = np.array([3, 4, 5])
b = np.array([0, 0, 0])

t1 = torch.from_numpy(a)
t2 = torch.from_numpy(b)
t3 = torch.zeros(2)
t4 = torch.ones(3)
print('t1:', t1)
print('------------------------')
print('t2:', t2)
print('------------------------')
print('t3:', t3)
print('------------------------')
print('t4:', t4)

res = torch.cat((t4, t3), 0)
print(res)

t1: tensor([3, 4, 5], dtype=torch.int32)
------------------------
t2: tensor([0, 0, 0], dtype=torch.int32)
------------------------
t3: tensor([0., 0.])
------------------------
t4: tensor([1., 1., 1.])
tensor([1., 1., 1., 0., 0.])


In [90]:
208/46

4.521739130434782

In [110]:
import numpy as np

a = np.random.randint(0,255,[79])

a

array([170, 124,  46, 141,  18, 112, 173, 162, 151,  62,  24, 170, 121,
        36, 105, 235,  98, 143, 163, 229, 163, 107, 137, 163, 141, 177,
       179, 158, 123, 143, 133, 175, 126, 132, 218, 116, 149,  10,  14,
        12, 126, 187, 149,  87, 119, 171, 217,  90, 106, 237,  79, 181,
       177,  93, 165, 240, 205, 145,  86,  73,  35, 210, 222, 221,  60,
       142, 135, 212, 215,  24,  43, 199, 129, 242, 246, 194,   6, 151,
        75])

In [111]:
def standardization(data):
    mu = np.mean(data, axis=0)
    sigma = np.std(data, axis=0)
    return (data - mu) / sigma

b = standardization(a)
b

array([ 0.53403124, -0.19091068, -1.42016002,  0.07700264, -1.86142902,
       -0.38002596,  0.58131006,  0.40795438,  0.23459871, -1.16800631,
       -1.76687138,  0.53403124, -0.2381895 , -1.57775609, -0.49034321,
        1.55840569, -0.60066046,  0.10852185,  0.42371399,  1.46384805,
        0.42371399, -0.458824  ,  0.01396421,  0.42371399,  0.07700264,
        0.64434849,  0.6758677 ,  0.34491595, -0.20667029,  0.10852185,
       -0.04907422,  0.61282927, -0.15939147, -0.06483383,  1.29049237,
       -0.31698754,  0.20307949, -1.98750587, -1.92446744, -1.95598666,
       -0.15939147,  0.80194456,  0.20307949, -0.77401614, -0.26970872,
        0.54979084,  1.27473276, -0.72673732, -0.47458361,  1.5899249 ,
       -0.90009299,  0.70738691,  0.64434849, -0.6794585 ,  0.4552332 ,
        1.63720372,  1.08561748,  0.14004106, -0.78977575, -0.99465064,
       -1.5935157 ,  1.16441552,  1.3535308 ,  1.33777119, -1.19952553,
        0.09276224, -0.01755501,  1.19593473,  1.24321355, -1.76